In [1]:
import numpy as np

In [2]:
with open('input/12-17-input','r') as f:
    data = [x.strip() for x in f][0]

In [3]:
data

'>>><<<<>><<<<>>>><<><<<<>><><<<><<<<>>><<<<><<>>>><<<<>><<<<>><<<>><>><<<>><>><<>>><<>><<<>>><<<<>>><<<<>>><<<<><<<<><>>><<<>>>><<>><<<><<<<><<<>>><<<<>>>><<<><<<><>>><<><>><<<<>>><>>>><<>>>><<<<><<<<><<<<>>><<>><><<<>><<<><<>>><<>>><>><>>><>>>><<<<>><<>>><<<>><<><<>><<<<>>><>><><>>>><<<<><>><<>><>>>><<>>><<>>><<>>><><<<<>>><<<>>><><<>>>><<<>><<<>>>><<>><<<>><<<>>>><>>>><<>><<<<>><<<>>><<>>><<>><<<>>>><<>><<<>><>>>><>><<<>>>><><<>>><<>>>><>><><<<<>>><<>><<>><<<<>><>>>><<<<>>>><<<>>><<<<>><<<<>>><<<<>>><<>>>><<>><>>><<><<<>>>><<<<>><<>><><<<<>>>><<<<><<<>><><<<><<>><>><<<<>><<><<<>><>><>><>>>><<<><>>><>>>><<<<>>><<<>>>><>>><>>><<<><<<>>><<<<>>><<><<<<>>>><<<<><<>>>><<>><<<<>>>><>>>><<>><<>><<><>>>><><><<<<>>><<<>>><<<<>>>><<>>><<<<>>>><><<><<<<>>><<<<>>>><<<>><>><<<<>>>><><>>>><>>><<><<<<>><<<<>>>><<>>><<>>>><>>><<>>><><<<<>>><<<>><>>>><<<><<<<>>>><<<<>><>><<>>>><<><>><>>><<>>>><<<<>>><<<<>>><<<>><<>><<>>><>><>><<<<>><<<>><><<<>><<<>>>><<<<><<<<>>><><<<><<>>><<<>>><<<<>>><<>>><<<><<>>>

In [4]:
data = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

In [5]:
rocks = [np.array([list('####')]), 
         np.array([list('.#.'), list('###'), list('.#.')]),
         np.array([list('..#'), list('..#'), list('###')]), 
         np.array([list('#'),list('#'),list('#'),list('#')]),
         np.array([list('##'), list('##')])]

In [6]:
rocks

[array([['#', '#', '#', '#']], dtype='<U1'),
 array([['.', '#', '.'],
        ['#', '#', '#'],
        ['.', '#', '.']], dtype='<U1'),
 array([['.', '.', '#'],
        ['.', '.', '#'],
        ['#', '#', '#']], dtype='<U1'),
 array([['#'],
        ['#'],
        ['#'],
        ['#']], dtype='<U1'),
 array([['#', '#'],
        ['#', '#']], dtype='<U1')]

In [310]:
chamber = np.array([list('-------')])

In [311]:
def show(c):
    for r in c:
        print(''.join(r))

In [312]:
show(chamber)

-------


In [313]:
def add_rock(c, rock):
    ###  Returns a new numpy array, with the added rock (and extra empty rows), 
    ###  and the row indicating the bottom of the newly added rock
    ch = np.vstack(([list('.......')], [list('.......')], [list('.......')], c.copy()))
    for row in rock[::-1]:
        r = '..' + ''.join(row) + '.'* (5-row.shape[0])
        ch = np.vstack((np.array([list(r)]), ch))
    return ch, (rock.shape[0] - 1, 2)

In [505]:
#  Must take into account the different rock shapes.  
#  Does it run into the side of the canyon and/or does it run into existing rock,
#  since it can drift left/right and then fall?  (So it can fall and then drift
#  under an overhanging "shelf", for example)   

def puff(c, rock_type, rock_bottom_left, direction):
    output = c.copy()
    
    #  Handle the rectangular-shaped rocks
    if rock_type in [0, 3, 4]:
        if (direction == '<' and rock_bottom_left[1] == 0) or\
           (direction == '>' and rock_bottom_left[1] == (7-rocks[rock_type].shape[1])) or\
           (direction == '<' and any(c[:rock_bottom_left[0]+1, rock_bottom_left[1]-1] == '#')) or\
           (direction == '>' and any(c[:rock_bottom_left[0]+1, rock_bottom_left[1]+rocks[rock_type].shape[1]] == '#')):
            return output, rock_bottom_left
         #  If we get here, then the rock can move
        if direction == '<': 
            output[0:rock_bottom_left[0]+1, rock_bottom_left[1]-1:rock_bottom_left[1]+rocks[rock_type].shape[1]-1] = '#'
            output[0:rock_bottom_left[0]+1, rock_bottom_left[1]+rocks[rock_type].shape[1]-1] = '.'
            return output, (rock_bottom_left[0], rock_bottom_left[1]-1)
        elif direction == '>': 
            output[0:rock_bottom_left[0]+1, rock_bottom_left[1]+1:rock_bottom_left[1]+rocks[rock_type].shape[1]+1] = '#'
            output[0:rock_bottom_left[0]+1, rock_bottom_left[1]] = '.'
            return output, (rock_bottom_left[0], rock_bottom_left[1]+1)
    
    #  Handle the "backwards L" rock
    if rock_type == 2:
        if (direction == '<' and (rock_bottom_left[1] == 0)) or\
           (direction == '>' and (rock_bottom_left[1] == (7-rocks[rock_type].shape[1])))  or\
           (direction == '<' and c[rock_bottom_left[0], rock_bottom_left[1]-1] == '#') or\
           (direction == '>' and any(c[0:rock_bottom_left[0]+1, rock_bottom_left[1]+rocks[rock_type].shape[1]] == '#')):
            return output, rock_bottom_left
        #  If we get here, then the rock can move
        if direction == '<':
            output[rock_bottom_left[0], rock_bottom_left[1]-1:rock_bottom_left[1]+rocks[rock_type].shape[1]-1] = '#'
            output[rock_bottom_left[0], rock_bottom_left[1]+rocks[rock_type].shape[1]-1] = '.'
            output[0:rock_bottom_left[0]+1, rock_bottom_left[1]+rocks[rock_type].shape[1]-2:rock_bottom_left[1]+rocks[rock_type].shape[1]-1] = '#'
            output[0:rock_bottom_left[0]+1, rock_bottom_left[1]+rocks[rock_type].shape[1]-1:rock_bottom_left[1]+rocks[rock_type].shape[1]] = '.'
            return output, (rock_bottom_left[0], rock_bottom_left[1] - 1)
        elif direction == '>':
            output[rock_bottom_left[0], rock_bottom_left[1]:rock_bottom_left[1]+rocks[rock_type].shape[1]+1] = '#'
            output[rock_bottom_left[0], rock_bottom_left[1]] = '.'
            output[0:rock_bottom_left[0]+1, rock_bottom_left[1]+rocks[rock_type].shape[1]:rock_bottom_left[1]+rocks[rock_type].shape[1]+1] = '#'
            output[0:rock_bottom_left[0], rock_bottom_left[1]+rocks[rock_type].shape[1]-1:rock_bottom_left[1]+rocks[rock_type].shape[1]] = '.'
            return output, (rock_bottom_left[0], rock_bottom_left[1] + 1)

In [506]:
rocks[2].shape

(3, 3)

In [507]:
c_new, rbl = add_rock(chamber, rocks[2])

In [508]:
c_new, rbl

(array([['.', '.', '.', '.', '#', '.', '.'],
        ['.', '.', '.', '.', '#', '.', '.'],
        ['.', '.', '#', '#', '#', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.'],
        ['-', '-', '-', '-', '-', '-', '-']], dtype='<U1'),
 (2, 2))

In [509]:
c_nn, rbl_n = puff(c_new, 2, rbl, '>')

In [510]:
c_nn, rbl_n

(array([['.', '.', '.', '.', '.', '#', '.'],
        ['.', '.', '.', '.', '.', '#', '.'],
        ['.', '.', '.', '#', '#', '#', '.'],
        ['.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.'],
        ['-', '-', '-', '-', '-', '-', '-']], dtype='<U1'),
 (2, 3))

In [511]:
blah, blahblah = puff( puff(c_nn, 2, rbl_n, '>')[0], 2, puff(c_nn, 2, rbl_n, '>')[1], '>')

In [514]:
blah

array([['.', '.', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '.', '#', '#', '#'],
       ['.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.'],
       ['-', '-', '-', '-', '-', '-', '-']], dtype='<U1')

In [512]:
blahblah

(2, 4)

In [515]:
puff (blah, 2, blahblah, '>')

(array([['.', '.', '.', '.', '.', '.', '#'],
        ['.', '.', '.', '.', '.', '.', '#'],
        ['.', '.', '.', '.', '#', '#', '#'],
        ['.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.'],
        ['-', '-', '-', '-', '-', '-', '-']], dtype='<U1'),
 (2, 4))

In [491]:
def fall(c, rock_bottom):
    output = c.copy()
    
    #  This should handle four out of the five rocks
    if np.any( (c[rock_bottom] == '#') & (c[rock_bottom+1] == '#') ):  #  bottom of rock hits a rock
        return output, rock_bottom
    #  This should handle the other rock, the "plus" rock
    if rock_bottom > 0:

IndentationError: expected an indented block (2504834557.py, line 8)

In [46]:
c_new, rb = add_rock(chamber, rocks[1])

In [32]:
(c_new[rb] == '#') & (c_new[rb+1] == '#')

array([False, False, False, False, False, False, False])

In [40]:
[(x, min(y for y in range(7) if c_new[x,y] == '#')) for x in range(rb+1)]

[(0, 3), (1, 2), (2, 3)]

In [52]:
puff( puff( puff( c_new, rb, '>'), rb, '>'), rb, '>')

[(0, 3), (1, 2), (2, 3)] [(0, 3), (1, 4), (2, 3)]
[(0, 4), (1, 3), (2, 4)] [(0, 4), (1, 5), (2, 4)]
[(0, 5), (1, 4), (2, 5)] [(0, 5), (1, 6), (2, 5)]


array([['.', '.', '.', '.', '.', '#', '.'],
       ['.', '.', '.', '.', '#', '#', '#'],
       ['.', '.', '.', '.', '.', '#', '.'],
       ['.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.'],
       ['-', '-', '-', '-', '-', '-', '-']], dtype='<U1')

In [ ]:
test[0:rb+1,-1]